<img style="float: right; width: 30%; height: 30%;" src="images/mongodb.jpg" />
# JADS Data Engineering: MongoDB Exercises

In this exercise we will work with MongoDB and cover the following subjects:

- Data model considerations
- Data insertion
- Simple queries
- More complex aggregations

The exercise will start by configuring the Jupyter environment and setting up a MongoDB in the cloud. Next we will download the dataset for this exercise and transform it to input that is suitable for Mongo. Finally we will insert this data, run some basic queries and some more complex aggregations.

### Setup

First of all we need access to a running MongoDB instance. While running MongoDB as a local process for testing and development purposes is not very complicated we will make use of a "Database-as-a-Service" provider for this exercise: 

1. Open a separate tab/window in your browser and browse to [https://mlab.com](https://mlab.com)
2. Sign up (for free) and log in to the service
3. mLab provides a free sandbox tier that is ideal for development and prototyping. Create a new Single-node deployment using the 'Sandbox' plan. Name it whatever you like (NB: use Amazon's EU Ireland region).
4. Create a user for the newly created database that you will use in this exercise

Next, we need to setup the Python environment so that the [Python driver](https://docs.mongodb.com/ecosystem/drivers/python/) is available. Execute the next cell to install the driver package:

In [ ]:
! conda install -y pymongo

Now we can verify that pymongo is installed correctly and that we can access our mLab database. We will do that by building a connection and querying the available collections (note that for a new database this will only return some system collections). In the next cells you will need to add the MongoDB URI that is shown on the information overview page (browse to your deployment). Note that you will need to add your username and password to the URI string. 

In [ ]:
import pymongo
from pymongo import MongoClient

# TODO replace the `< >` with values for your deployment
db_URI = 'mongodb://<user>:<pass>@<host>.mlab.com:<port>/<dbname>' 
db_name = '<dbname>'

client = MongoClient(db_URI)
db = client[db_name]
res = db.collection_names()

print(res)

If everything checks out proceed to the next section. Make sure that you have the [pymongo API documentation](http://api.mongodb.com/python/current/api/pymongo/) and the [pymongo tutorial](http://api.mongodb.com/python/current/tutorial.html) open in your browser so that you can reference it during the exercises.

### The dataset

Today, we will be using the [MovieLens 100K Dataset][1-1], a dataset containing movie ratings from the [MovieLens](https://movielens.org) website.

First, download and unzip the 100K dataset. You will only need to consider the following files from the archive:

 - u.data
 - u.item
 - u.genre
 - u.user
 - u.occupation

[1-1]: http://grouplens.org/datasets/movielens/100k/

In [ ]:
%%bash
wget -q http://files.grouplens.org/datasets/movielens/ml-100k.zip && unzip ml-100k.zip

The MovieLens data was collected through the MovieLens web site (movielens.umn.edu) during the seven-month period from September 1997 through April 1998. The data set consists of:

- 100,000 ratings (1-5) from 943 users on 1682 movies. 
- Each user has rated at least 20 movies. 
- Simple demographic info for the users (age, gender, occupation, zip)

This data is organized in five files:

- __u.data:__ The 100,000 ratings. Users and items are numbered consecutively from 1. The data is randomly ordered. This is a tab separated list of user id | item id | rating | timestamp. The time stamps are unix seconds since 1/1/1970 UTC   

- __u.item:__ Information about the items (movies); this is a pipe separated list of movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western | The last 19 fields are the genres, a 1 indicates the movie is of that genre, a 0 indicates it is not; movies can be in several genres at once. The movie ids are the ones used in the u.data data set.

- __u.genre:__ A list of the genres.

- __u.user:__ Demographic information about the users; this is a pipe separated list of user id | age | gender | occupation | zip code The user ids are the ones used in the u.data data set.

- __u.occupation:__ A list of the occupations.

Print the first two records of each of the files to better understand the data:

In [ ]:
%%bash
echo "u.data:"
head -n 2 ml-100k/u.data
echo ""

echo "u.item:"
head -n 2 ml-100k/u.item
echo ""

echo "u.genre:"
head -n 2 ml-100k/u.genre
echo ""

echo "u.user:"
head -n 2 ml-100k/u.user
echo ""

echo "u.occupation:"
head -n 2 ml-100k/u.occupation

### Exercise I: data model

If you consider the data in more detail and think about how they would fit in a (simple)Entity-Relationship model you would probably come up with something not unlike the following:

- Strong entities: Movie (item) and user
- Relationships: A user rates a movie

Or graphically (some attributes intentionally left out):

</br></br>
<img style="width: 75%; height: 75%;" src="images/simpleer3.png" />
</br></br>

You might model the genre and occupation attributes as weak entities with a relationships. Translating this to a relational database model is rather trivial. 

For this first exercise think about how you would translate this model to the document store MongoDB. Think about the different possibilities of storing the information: from fully denormalized where each record stores all the data, concerning that record, to a more normalized approach with separate collections modelling separate 'entities'. 

__Assignment:__ Describe how you would translate the data to documents in a document store and what information your documents and collections will store. Describe the pros and cons of your approach.

TODO: ADD YOUR DESCRIPTION HERE

### Exercise IIa: Loading the data in MongoDB

Now you will have to provide functionality to load the datafiles into Mongo. First you will have to engage in the less sexy part of data science and convert the MovieLens data to collections of JSON documents to insert in Mongo. Consider the following user and item examples:

    {
        "id": 474
        "title": "Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)",
        "release_date": "01-Jan-1963",
        "genres": [
            "Sci-Fi",
            "War"
        ]
    }
     
    {
        "id": 73
        "age": 24,
        "gender": "M",
        "occupation": "student",
        "zip": "41850"
    }

If you don't know how to approach this, use the following hint:

Gur rnfvrfg nccebnpu vf gb ernq va gur hfre naq vgrz qngn va zrzbel. Perngr gjb ybbxhc gnoyrf (unfuznc be qvpgvbanel). Arkg ernq va gur engvatf naq hfr gur ybbxhc gnoyrf gb bhgchg qrabeznyvfrq qngn, ntnva va n yvfg be qvpg.

As a convenience we have already provided a genre lists you can use in your function(s). If, after half an hour you are still stuck with this call for help.

__Assignment:__ Write functions to convert the files to collections of dicts. Create these collections.

In [ ]:
# TODO Convert the data files to collections of JSON records (dicts in Python will do).

genre_list = ["unknown", "Action", "Adventure", "Animation", "Children's",
              "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir",
              "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller",
              "War", "Western"]

### Exercise IIb: Loading the data in MongoDB

MongoDB supports bulk writes; read the pymongo documentation of the [`bulk_write`](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.bulk_write) function to see how the function is called. Use the collections you created in exercise IIa to insert the data in MongoDB. You can reuse the db object from the initial setup.

Once the data is in demonstrate this by selecting one item from each of the collections you created. You can use the 
[`find`](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find) or [`find_one`](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find_one) functions for this.

__Assignment:__ Bulk insert the collections you created. Demonstrate by selecting one item from each of the document types or collections you created.

In [ ]:
# TODO Load your collections of dicts into your mLab MongoDB deployment

### Exercise III: Queries and aggregations

Once the data is imported you can use the MongoDB data manipulation commands to browse your database. Try some simple queries first and then write and perform queries that answer the following questions:

 - Which movie has received the most ratings?
 - What is the average rating for the movie 'Star Wars'?
 - What is the occupation and age of the user that has given the most ratings?
 
You will need to use the MongoDB aggregation framework to answer these questions (perhaps combined with client side code). The new aggregation framework offers the most flexibility and performance. Before starting the exercise look at the worked examples in the [documentation](http://api.mongodb.com/python/current/examples/aggregation.html).

__Extra__: Based on the queries above, and the general structure of the database, can you identify any columns that will need indexes? Does the data model you chose impact the way you structure your queries? Does it impact any client side processing?

Finally if you have time and you want to practice some more advanced queries, try answering the following
questions:

 - How many different genres does a movie have on average?
 - What is the male/female distribution over both the userbase and the ratings?
 - What is the most controversial movie (has the highest variance in its
   ratings)?

__Extra__: Plot the distributions using e.g. Matplotlib or Seaborn

__Assignment:__ Try to answer at least the first three questions

In [ ]:
# TODO implement your queries